In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

sns.set()
%matplotlib inline

C:\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
submission = pd.read_csv("./data/submission.csv")

In [22]:
print(train.shape)
train.head(5)

(1459, 11)


,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0


In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1459 non-null   int64  
 1   hour                    1459 non-null   int64  
 2   hour_bef_temperature    1457 non-null   float64
 3   hour_bef_precipitation  1457 non-null   float64
 4   hour_bef_windspeed      1450 non-null   float64
 5   hour_bef_humidity       1457 non-null   float64
 6   hour_bef_visibility     1457 non-null   float64
 7   hour_bef_ozone          1383 non-null   float64
 8   hour_bef_pm10           1369 non-null   float64
 9   hour_bef_pm2.5          1342 non-null   float64
 10  count                   1459 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB


In [26]:
train

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,21.0
1455,2175,3,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,20.0
1456,2176,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,2178,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0


In [36]:
from sklearn.model_selection import train_test_split

features = train.dropna().loc[:,:"hour_bef_pm2.5"]
target = train.dropna()['count']

X_train,X_test,y_train,y_test = train_test_split(features,target,test_size=0.2)

In [38]:
# baseline model

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score

base_model = LinearRegression()

base_model.fit(X_train,y_train)

base_pred = base_model.predict(X_test)

base_scores = cross_val_score(base_model, X_train, y_train, scoring='neg_mean_squared_error', cv=3)
base_rmse_scores = np.sqrt(-1*base_scores)
base_rmse = np.mean(base_rmse_scores)
base_r2 = r2_score(base_pred,y_test)

print("base model rmse : ",base_rmse)
print("base model r2 : ",base_r2)

base model rmse :  53.986180978895824
base model r2 :  0.382824723244777


In [39]:
train

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,21.0
1455,2175,3,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,20.0
1456,2176,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,2178,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0


In [52]:
from sklearn.preprocessing import StandardScaler

def data_pre(df):
    
    sc = StandardScaler()
    
    features = train.dropna().loc[:,:"hour_bef_pm2.5"]
    target = train.dropna()['count']
    
    features = features.drop(['id'],axis=1)
    features = pd.get_dummies(features, columns=['hour'])
    
    features = sc.fit_transform(features)
    
#     X_train,X_test,y_train,y_test = train_test_split(features,target,test_size=0.2)
    
    return features
#     return X_train,X_test,y_train,y_test
    
    

In [58]:
sc = StandardScaler()

sc.fit_transform(train)

array([[-1.74754458,  1.22919048, -0.07970308, ...,  0.59293487,
         0.18173227, -0.72107676],
       [-1.74279114,  0.21769081,  0.64585444, ...,  0.49847469,
         0.65767121,  0.61058738],
       [-1.74120666, -0.79380886, -0.53794994, ..., -0.79248114,
        -0.7701456 , -0.99951563],
       ...,
       [ 1.69552827, -0.93830881,  0.3021693 , ..., -0.8554546 ,
        -0.63416305, -1.04793978],
       [ 1.69869723,  1.37369043,  0.76041615, ...,  0.43550123,
         0.3857061 ,  1.30063153],
       [ 1.70028171,  0.79569062,  0.83679063, ..., -0.60356078,
        -0.90612815,  0.7437538 ]])

In [55]:
ss = data_pre(train)

In [56]:
ss

(array([[-0.10896712,  5.67450438, -0.71609016, ..., -0.21562305,
         -0.21752842, -0.21562305],
        [ 0.62256362, -0.17622684, -0.7884346 , ..., -0.21562305,
         -0.21752842, -0.21562305],
        [-0.57098652, -0.17622684, -1.29484572, ..., -0.21562305,
         -0.21752842, -0.21562305],
        ...,
        [ 0.27604906, -0.17622684, -0.42671238, ..., -0.21562305,
         -0.21752842, -0.21562305],
        [ 0.73806847, -0.17622684,  0.87548763, ...,  4.63772299,
         -0.21752842, -0.21562305],
        [ 0.8150717 , -0.17622684,  0.44142096, ..., -0.21562305,
         -0.21752842, -0.21562305]]), 0        49.0
 1       159.0
 2        26.0
 3        57.0
 4       431.0
         ...  
 1454     21.0
 1455     20.0
 1456     22.0
 1457    216.0
 1458    170.0
 Name: count, Length: 1328, dtype: float64)